# Bayesian localization of Neptune

In [ ]:
%matplotlib inline

import cv2

import numpy as np
import numpy.linalg as la 
import matplotlib.pyplot as plt
from scipy.integrate import odeint

from umucv.kalman import kalman, ukf
from umucv.htrans import rotation

from matplotlib.patches import Ellipse

In [ ]:
ua = 150e9
yr = 365*24*60*60

In [ ]:
def accel(x,m):
    n = len(m)
    a = np.zeros([n,3])
    for k in range(n):
        for j in range(n):
            if k != j:
                r = x[j]-x[k]
                r3 = la.norm(r)**3
                a[k] += m[j] / r3 * r
    return a

In [ ]:
def nbody(r0,v0,m,t):
    n = len(m)

    def xdot(z,t):
        #print(len(z))
        global count
        count += 1
        r = z[:3*n].reshape(-1,3)
        v = z[3*n:]
        a = accel(r,m).flatten()
        return np.concatenate([v,a])

    s0 = np.concatenate([r0.flatten(),v0.flatten()])
    #print(s0)
    s = odeint(xdot,s0,t)

    return [(s[:3*n].reshape(-1,3), s[3*n:].reshape(-1,3)) for s in s]

In [ ]:
def cart2sph(v):
    x,y,z = v
    r = np.sqrt(x**2+y**2+z**2)
    t = np.arccos(z/r)
    f = np.arctan2(y,x)
    return np.array([r,t,f])

In [ ]:
def view(v):
    s = cart2sph(v)
    return np.degrees(s[1:])

https://en.wikipedia.org/wiki/Standard_gravitational_parameter

https://downloads.rene-schwarz.com/download/M002-Cartesian_State_Vectors_to_Keplerian_Orbit_Elements.pdf

## Uranus - Neptune

In [ ]:
datar = np.array(np.matrix("""
0      0   0   0   0        0    1.327E20    ;
0.466  0   0   0   47E3     0    2.203E13    ;
0.723  0   0   0   35E3     0    3.249E14    ;
1      0   0   0   30.0E3   0    3.986E14    ;
1.0026 0   0   0   31.0E3   0    4.905E12    ;
1.52   0   0   0   24E3     0    4.283E13    ;
5.2    0   0   0   13E3     0    1.267E17    ;
9.5    0   0   0   9.7E3    0    3.793E16    ;
19     0   0   0   6.8E3    0    5.794E15    ;
30     0   0   0   5.43E3   0    6.837E15    """))


#data = data[[0,8,9,3]]
data = datar[[8,9,0]]

r0 = data[:,:3]
v0 = data[:,3:6] * yr    / ua
mu = data[:,6]   * yr**2 / ua**3


r0[0] = rotation((0,0,1),np.radians(-30)) @ r0[0]

v0[0] = rotation((0,0,1),np.radians(-30)) @ rotation((1,0,0),np.radians(-20)) @ v0[0]
v0[1] =                                     rotation((1,0,0),np.radians( 10)) @ v0[1]

In [ ]:
mu

In [ ]:
dt = 3
N = 160//3

count = 0
paths = np.array([x[0] for x in nbody(r0,v0,mu,np.arange(N+1)*dt)]).transpose(1,2,0)
print(count)

In [ ]:
plt.figure(figsize=(10,10))
for p in paths:
    plt.plot(p[0],p[1],'.-')
plt.axis('equal');

In [ ]:
relpaths = np.apply_along_axis(view,-1,np.delete((paths - paths[-1]),-1,axis=0).transpose(0,2,1)).transpose(0,2,1)

In [ ]:
plt.figure(figsize=(10,5))
for p in relpaths[0:]:
    plt.plot(p[1],p[0],'.-',markersize=3)
plt.axis('equal'); plt.grid()

In [ ]:
mu_2 = mu.copy()
mu_2[1] = mu[1]/1e6
count = 0
paths2 = np.array([x[0] for x in nbody(r0,v0,mu_2,np.arange(N+1)*dt)]).transpose(1,2,0)
print(count)

In [ ]:
plt.figure(figsize=(10,10))
for p in paths2:
    plt.plot(p[0],p[1],'.-')
plt.axis('equal');

In [ ]:
plt.figure(figsize=(10,10))
for p in paths:
    plt.plot(p[0],p[1],'.-')
for p in paths2:
    plt.plot(p[0],p[1],'.-')
plt.axis('equal');

In [ ]:
relpaths2 = np.apply_along_axis(view,-1,np.delete((paths2 - paths2[-1]),-1,axis=0).transpose(0,2,1)).transpose(0,2,1)

In [ ]:
plt.figure(figsize=(10,10))
for p in relpaths[0:1]:
    plt.plot(p[1],p[0],'.-',markersize=5,lw=.1)
for p in relpaths2[0:1]:
    plt.plot(p[1],p[0],'.-',markersize=5,lw=.1)
plt.axis([88,89,107,108]); plt.grid()

In [ ]:
col = ['blue','green','red']
c = 0
plt.figure(figsize=(10,10))
for p in (relpaths-relpaths2)[0:].transpose(0,2,1)*60*60:
    for e in p:
        plt.plot([0,e[1]],[0,e[0]],'.-',color=col[c],lw=0.1)
    c += 1
for p in (relpaths-relpaths2)[0:1]*60*60:
    plt.plot(p[1],p[0])
plt.axis('equal'); plt.grid(); plt.title('error (arcsec)');

In [ ]:
err = np.apply_along_axis(la.norm,1,paths-paths2)

In [ ]:
plt.plot(np.arange(len(err[1]))*3,err[1]);

In [ ]:
for p in (relpaths-relpaths2)[0:1]*60*60:
    plt.plot(np.arange(len(err[1]))*3,p[1])

In [ ]:
paths.transpose(0,2,1)[:,0,:]

In [ ]:
r0

In [ ]:
def st2vec(pv):
    r,v,m = pv
    return np.concatenate([r.flatten(),v.flatten(),m])

def vec2st(st):
    n = len(st)//7
    r = st[0:3*n].reshape(-1,3)
    v = st[3*n:6*n].reshape(-1,3)
    m = st[-n:]
    return r,v,m

def obs1(st):
    #return st[[0,1,2,6,7,8,9,10,11,15,16,17]]
    return st[[0,1,2,6,7,8]]


In [ ]:
st2vec((r0,v0,mu))

In [ ]:
vec2st(st2vec((r0,v0,mu)))

In [ ]:
paths[:,:,0]

In [ ]:
r0

In [ ]:
v0

In [ ]:
count = 0
paths = np.array([st2vec((x,v,mu)) for x,v in nbody(r0,v0,mu,np.arange(N+1)*dt)])
print(count)
print(paths.shape)

In [ ]:
paths[0]

In [ ]:
obs = np.apply_along_axis(obs1,1,paths)
print(obs.shape)

In [ ]:
paths

In [ ]:
paths[0]

In [ ]:
def f(x):
    r,v,m = vec2st(x)
    _,(r,v) = nbody(r,v,m,[0,dt])
    return st2vec((r,v,m))

def h(x):
    return obs1(x)

def b(x):
    return 0

In [ ]:
f(paths[10]) - paths[11]

In [ ]:
paths[2]

In [ ]:
v0

In [ ]:
sigmaP = 1e-5
sigmaM = 1e-9
sigmaZ = 1e-5


mU  = paths[0].copy()
P  = sigmaP**2*np.eye(len(mU))

mU[3] = 40
P[3,3] = 10**2

mU[4] += -5
P[4,4] = 10**2

mU[5] = 2
P[5,5] = 2**2


mU[12] = 0
P[12,12] = 0.2**2


mU[13] = 1
P[13,13] = 0.2**2

mU[14] = 0
P[14,14] = 0.2**2


mU[19] = 1.5e-3
P[19,19] = 0.5e-3 **2


Q = sigmaM**2 * np.eye(len(mU))
R = sigmaZ**2 * np.eye(len(obs[0]))

res = [(mU,P,obs1(mU))]
epath = [mU]
preds = [obs1(mU)]

In [ ]:
N = 15

for z in obs[1:N+1]:
    mU,P,pred = ukf(mU,P,f,Q,b,0,z,h,R)
    print(pred.shape)
    res += [(mU,P,pred)]
    epath += [mU]
    preds += [pred]

epath = np.array(epath).T
print(epath.shape)

In [ ]:
xe = [mu[3] for mu,_,_ in res]             # coordenada x estimada
xu = [2*np.sqrt(P[3,3]) for _,P,_ in res]  # su incertidumbre

ye = [mu[4] for mu,_,_ in res]             # lo mismo para y
yu = [2*np.sqrt(P[4,4]) for _,P,_ in res]

fig,ax = plt.subplots(figsize=(12,12))

for k in range(len(xe)):
    ax.add_patch(Ellipse(xy=(xe[k],ye[k]), width=xu[k], height=yu[k], angle = 0, alpha=0.2))

plt.plot(paths[:N+1,0],paths[:N+1,1],'-',color='gray',lw=0.5)
plt.plot(paths[:N+1,3],paths[:N+1,4],'-',color='gray',lw=0.5)
plt.plot(epath[0],epath[1],'.-')
plt.plot(epath[3],epath[4],'.-')
plt.axis('equal');

In [ ]:
errs=np.abs(obs[0:N+1].T - np.array(preds).T)

col = ['blue','blue','blue', 'green','green','green', 'gray', 'gray', 'gray', 'gray', 'gray', 'gray']
c = 0
for e in errs:
    plt.plot(e,color=col[c])
    c +=1

In [ ]:
plt.plot(epath[19],label='estimated');
plt.plot([0,N],[mu[1],mu[1]],label='true');
plt.title('Neptune mass');
plt.xlabel('steps'); plt.ylabel('GM (UA-yr)')
plt.legend();

In [ ]:
v = []
for x in epath.T:
    n = x[3:6]
    s = x[6:9]
    v += [view(n-s)]
v = np.array(v).T
v.shape

w = []
for x in paths[0:N+1]:
    n = x[3:6]
    s = x[6:9]
    w += [view(n-s)]
w = np.array(w).T
w.shape

plt.figure(figsize=(10,5))
plt.plot(v[1],v[0],'.-')
plt.plot(w[1],w[0],'.-')
plt.axis('equal'); plt.grid()

In [ ]:
plt.plot(np.apply_along_axis(la.norm,1,(v-w).T));
plt.grid();
plt.title('heliocentric estimation error');

In [ ]:
plt.plot(np.sqrt(np.diag(P)));

In [ ]:
plt.plot(np.sqrt(np.diag(P)[:3]),'.-');

In [ ]:
for k in range(10):
    mU,P,pred = ukf(mU,P,f,Q,b,0,None,h,R)

In [ ]:
plt.bar(np.arange(21),np.sqrt(np.diag(P)));

In [ ]:
plt.bar(np.arange(3),np.sqrt(np.diag(P)[:3]));

In [ ]:
plt.bar(np.arange(21),np.sqrt(np.diag(P)));

In [ ]:
plt.bar(np.arange(3),np.sqrt(np.diag(P)[:3]));

## Jupiter - Saturn

In [ ]:
datar = np.array(np.matrix("""
0      0   0   0   0        0    1.327E20    ;
0.466  0   0   0   47E3     0    2.203E13    ;
0.723  0   0   0   35E3     0    3.249E14    ;
1      0   0   0   30.0E3   0    3.986E14    ;
1.0026 0   0   0   31.0E3   0    4.905E12    ;
1.52   0   0   0   24E3     0    4.283E13    ;
5.2    0   0   0   13E3     0    1.267E17    ;
9.5    0   0   0   9.7E3    0    3.793E16    ;
19     0   0   0   6.8E3    0    5.794E15    ;
30     0   0   0   5.43E3   0    6.837E15    """))


#data = data[[0,8,9,3]]
data = datar[[6,7,0,3]]

r0 = data[:,:3]
v0 = data[:,3:6] * yr    / ua
mu = data[:,6]   * yr**2 / ua**3


r0[0] = rotation((0,0,1),np.radians(-30)) @ r0[0]

v0[0] = rotation((0,0,1),np.radians(-30)) @ rotation((1,0,0),np.radians(0)) @ v0[0]
v0[1] =                                     rotation((1,0,0),np.radians(0)) @ v0[1]

In [ ]:
dt = 1/4
N = 100*4//1
N = 80

count = 0
paths = np.array([x[0] for x in nbody(r0,v0,mu,np.arange(N+1)*dt)]).transpose(1,2,0)
print(count)

In [ ]:
plt.figure(figsize=(10,10))
for p in paths:
    plt.plot(p[0],p[1],'.-',lw=0.3)
plt.axis('equal');

In [ ]:
relpaths = np.apply_along_axis(view,-1,np.delete((paths - paths[-1]),-1,axis=0).transpose(0,2,1)).transpose(0,2,1)

In [ ]:
plt.figure(figsize=(10,5))
for p in relpaths[0:]:
    plt.plot(p[1],p[0],'.-',markersize=3)
plt.axis('equal'); plt.grid()

In [ ]:
mu_2 = mu.copy()
mu_2[1] = mu[1]/1e6
mu_2[0] = mu[0]/1e6
count = 0
paths2 = np.array([x[0] for x in nbody(r0,v0,mu_2,np.arange(N+1)*dt)]).transpose(1,2,0)
print(count)

In [ ]:
plt.figure(figsize=(10,10))
for p in paths2:
    plt.plot(p[0],p[1],'.-',lw=0.3)
plt.axis('equal');

In [ ]:
plt.figure(figsize=(10,10))
for p in paths:
    plt.plot(p[0],p[1],'.',markersize=5)
for p in paths2:
    plt.plot(p[0],p[1],'.',markersize=2, color='gray')
plt.axis('equal');

In [ ]:
relpaths2 = np.apply_along_axis(view,-1,np.delete((paths2 - paths2[-1]),-1,axis=0).transpose(0,2,1)).transpose(0,2,1)

In [ ]:
def dif(x):
    return 180 - np.remainder(180 + x, 360)

In [ ]:
col = ['blue','green','red']
c = 0
plt.figure(figsize=(10,10))
if False:
    for p in (relpaths-relpaths2)[0:].transpose(0,2,1)*60:
        for e in p:
            plt.plot([0,e[1]],[0,e[0]],'.-',color=col[c],lw=0.1)
        c += 1
for p in dif(relpaths-relpaths2)*60:
    plt.plot(p[1],p[0],'.-')
plt.axis('equal'); plt.grid(); plt.title('error (arcmin)');

In [ ]:
err = np.apply_along_axis(la.norm,1,paths-paths2)

In [ ]:
plt.plot(np.arange(len(err[1]))*3,err[1]);

In [ ]:
for p in dif(relpaths-relpaths2)[0:2]*60:
    plt.plot(np.arange(len(err[1]))*3,p[1])

In [ ]:
def a2sc(a):
    ad = np.degrees(a)
    return np.array([np.cos(ad), np.sin(ad)])

In [ ]:
def obs2(st):
    ju = st[0:3]
    sa = st[3:6]
    su = st[6:9]
    ea = st[9:12]
    v1 = view(ju-ea)
    v2 = view(sa-ea)
    v3 = view(su-ea)
    return  np.concatenate([v1,v2,v3])
    #return np.concatenate([ a2sc(x) for x in np.concatenate([v1,v2,v3]) ])

In [ ]:
def obs2(st):
    return st[0:12]
    #return np.concatenate([ a2sc(x) for x in np.concatenate([v1,v2,v3]) ])

In [ ]:
r0

In [ ]:
v0

In [ ]:
count = 0
paths = np.array([st2vec((x,v,mu)) for x,v in nbody(r0,v0,mu,np.arange(N+1)*dt)])
print(count)
print(paths.shape)

In [ ]:
paths[0]

In [ ]:
obs = np.apply_along_axis(obs2,1,paths)
print(obs.shape)

In [ ]:
def f(x):
    r,v,m = vec2st(x)
    _,(r,v) = nbody(r,v,m,[0,dt])
    return st2vec((r,v,m))

def h(x):
    return obs2(x)

def b(x):
    return 0

In [ ]:
v0

In [ ]:
mu

In [ ]:
paths[0]

In [ ]:
sigmaP = 1e-6
sigmaM = 1e-9
sigmaZ = 1e-6


mU  = paths[0].copy()
P  = sigmaP**2*np.eye(len(mU))

for k in [0,1,3,4]:
    P[k,k] = 0.5**2

#mU[3] += 2
#mU[1] -= 2
    
mU[24] = 2e-2
mU[25] = 2e-2

P[24,24] = 5e-3 ** 2
P[25,25] = 5e-3 ** 2

for k in [12,13,15,16]:
    P[k,k] = 0.5**2


Q = sigmaM**2 * np.eye(len(mU))
R = sigmaZ**2 * np.eye(len(obs[0]))

res = [(mU,P,obs2(mU))]
epath = [mU]
preds = [obs2(mU)]

In [ ]:
N = 5

for z in obs[1:N+1]:
    mU,P,pred = ukf(mU,P,f,Q,b,0,z,h,R)
    print(pred.shape)
    res += [(mU,P,pred)]
    epath += [mU]
    preds += [pred]

for k in range(0):
    mU,P,pred = ukf(mU,P,f,Q,b,0,None,h,R)
    print(pred.shape)
    res += [(mU,P,pred)]
    epath += [mU]
    preds += [pred]


epath = np.array(epath).T
print(epath.shape)

In [ ]:
fig,ax = plt.subplots(figsize=(12,12))

xe = [mu[3] for mu,_,_ in res]             # coordenada x estimada
ye = [mu[4] for mu,_,_ in res]             # lo mismo para y


eP34 = [la.eigh(P[3:5,3:5]) for _,P,_ in res]
d1 = [2*np.sqrt(x[0][0]) for x in eP34]
d2 = [2*np.sqrt(x[0][1]) for x in eP34]
an = [np.degrees(np.arctan2(x[1][0][1],x[1][0][0])) for x in eP34]


for k in range(len(xe)):
    ax.add_patch(Ellipse(xy=(xe[k],ye[k]), width=d1[k], height=d2[k], angle = an[k], alpha=0.2))

xe = [mu[0] for mu,_,_ in res]             # coordenada x estimada
ye = [mu[1] for mu,_,_ in res]             # lo mismo para y

eP34 = [la.eigh(P[0:2,0:2]) for _,P,_ in res]
d1 = [2*np.sqrt(x[0][0]) for x in eP34]
d2 = [2*np.sqrt(x[0][1]) for x in eP34]
an = [np.degrees(np.arctan2(x[1][0][1],x[1][0][0])) for x in eP34]

for k in range(len(xe)):
    ax.add_patch(Ellipse(xy=(xe[k],ye[k]), width=d1[k], height=d2[k], angle = an[k], alpha=0.2))

plt.plot(paths[:N+1,6],paths[:N+1,7],'.-',color='gray',lw=0.5)
plt.plot(paths[:N+1,9],paths[:N+1,10],'.-',color='gray',lw=0.5)
plt.plot(paths[:N+1,0],paths[:N+1,1],'-',color='gray',lw=0.5)
plt.plot(paths[:N+1,3],paths[:N+1,4],'-',color='gray',lw=0.5)
plt.plot(epath[0],epath[1],'.-')
plt.plot(epath[3],epath[4],'.-')
plt.axis('equal');

In [ ]:
errs=np.abs(obs[0:N+1].T - np.array(preds).T)

col = ['blue','blue','blue', 'green','green','green', 'gray', 'gray', 'gray', 'gray', 'gray', 'gray']
c = 0
for e in errs:
    plt.plot(e)#,color=col[c])
    c +=1

In [ ]:
plt.plot(epath[24],label='estimated');
plt.plot([0,N],[mu[0],mu[0]],label='Jupiter');

plt.plot(epath[25],label='estimated');
plt.plot([0,N],[mu[1],mu[1]],label='Saturn');

plt.plot([x[25] + 2*np.sqrt(P[25,25]) for x,P,_ in res],color='gray');
plt.plot([x[25] - 2*np.sqrt(P[25,25]) for x,P,_ in res],color='gray');
plt.plot([x[24] + 2*np.sqrt(P[24,24]) for x,P,_ in res],color='gray');
plt.plot([x[24] - 2*np.sqrt(P[24,24]) for x,P,_ in res],color='gray');

plt.title('estimated masses');
plt.xlabel('steps'); plt.ylabel('GM (UA-yr)')
plt.legend();